In [1]:
#!pip install tweet-preprocessor

In [528]:
import preprocessor as p
import pandas as pd
import json
import re

In [529]:
pd.options.display.max_colwidth = 75

In [703]:
pwd

'/home/jovyan/capstone-35/gulf_twitter_pickled'

In [704]:
cd ../gulf_twitter_raw/gulf_timelines/

/home/jovyan/capstone-35/gulf_twitter_raw/gulf_timelines


In [705]:
ls

user_timeline_Basbosah.jsonl


### Turn query documents (tweets) into generators

In [124]:
def load_tweets(file):
    with open(file, 'r') as f:
        tweets = (json.loads(line) for line in f.readlines())
    return tweets

In [706]:
query_gen = load_tweets('d')

### Turn query generators into dataframes

In [707]:
def tweet_to_df(tweets):
    data = {'text': [], 'screen_name': [], 'geo': [],
       }
    
    for t in tweets:
    
        data['text'].append(t['text'])
        data['screen_name'].append(t['user']['screen_name'])
        data['geo'].append(t['user']['location'])
    
    return pd.DataFrame(data)

In [708]:
query_df = tweet_to_df(query_gen)

In [709]:
query_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3183 entries, 0 to 3182
Data columns (total 3 columns):
geo            3183 non-null object
screen_name    3183 non-null object
text           3183 non-null object
dtypes: object(3)
memory usage: 74.7+ KB


### Clean tweets (removes emojis, links, special characters, and ASCII chars/digits from tweet text)

Using regular expressions such as `[0-9A-Za-z:&!/._?)؟'+-=—]` can filter out much of the unwanted data and retain Arabic characters but still seems to also retain a more recently released generation of emojis. Try updating tweet-preprocessor to version 0.5.0 or using another tweet processing package. 

Different scenarios will require different regex combinations. For example, the below approach wouldn't be of much use if you're building English dialect classifiers.

In [710]:
def cleaner(message):
    message = re.sub("([0-9A-Za-z:&!/._?)؟'+-=—])", " ", message)
    return message

In [711]:
cleaned_text = []

for tweet in query_df['text']:
    cleaned_text.append(p.clean(cleaner(tweet)))

query_df['cleaned_text']=cleaned_text

In [712]:
cleaned_name = []

for tweet in query_df['screen_name']:
    cleaned_name.append(p.clean(tweet))

query_df['cleaned_name']=cleaned_name

In [713]:
cleaned_geo = []

for tweet in query_df.geo:
    cleaned_geo.append(p.clean(str(tweet)))

query_df['cleaned_geo']=cleaned_geo


In [714]:
query_df.sample(4)

,geo,screen_name,text,cleaned_text,cleaned_name,cleaned_geo
700,East - مملكتي,Basbosah,@albassamah Whose this 🙄🙄🙄,,Basbosah,East - مملكتي
860,East - مملكتي,Basbosah,Oh Winter what surprises do you carry for us 🤓its getting warmer 😉,🤓,Basbosah,East - مملكتي
3130,East - مملكتي,Basbosah,اللي يبينا عيت النفس تبغيه ✋🏼 \nواللي نبيه عيا البخت لا يجيبه 👊🏼,اللي يبينا عيت النفس تبغيه واللي نبيه عيا البخت لا يجيبه,Basbosah,East - مملكتي
3065,East - مملكتي,Basbosah,@nawal66 شكله بيرد عليك : إنت ع إيش مستعجله 😜,شكله بيرد عليك إنت ع إيش مستعجله,Basbosah,East - مملكتي


### Remove duplicated tweets

In [715]:
query_df['cleaned_text'].duplicated().sample(5)

681     False
1279    False
570     False
2661    False
1256    False
Name: cleaned_text, dtype: bool

In [716]:
query_cleaned = query_df.drop_duplicates(['cleaned_text'], keep=False)
query_cleaned = query_cleaned.drop(['screen_name', 'text', 'geo'], axis=1)

In [717]:
query_cleaned.sample(5)

,cleaned_text,cleaned_name,cleaned_geo
642,من الخبل اللي بيقلب لونه بكلوركس ياربيييه,Basbosah,East - مملكتي
2363,و ع ناااار انتظر الحلقه الاخيره من قيم اوف ثرونز تخلص داونلوود يا الله ...,Basbosah,East - مملكتي
2325,وساعه أنتظر وشو اللي كومز ات ناديت ماعندهم ماعند جدي واجد يقهر الفلم سخيف,Basbosah,East - مملكتي
2305,نحمد الله أن مكّن لهذه البلاد خدمة ضيوف الرحمن، ونحمده أن بلغنا عيد الأ...,Basbosah,East - مملكتي
2234,قبلوا تراب الوطن هذا الزمان أخضر واشكروا مجزي النعم يزيدكم أكثر كرامةٍ ...,Basbosah,East - مملكتي


In [718]:
query_cleaned.fillna('None')

,cleaned_text,cleaned_name,cleaned_geo
0,فديت اهل الإمارات 🤗🤗🤗,Basbosah,East - مملكتي
1,تخيلي مليار نسمة وتحبيلك واحد ياختي اقل حاجة حبي عشرين,Basbosah,East - مملكتي
2,استقبل بشار الأسد اليوم الرئيس الروسي بوتن بدا كل شيء طبيعي ولكن رواد و...,Basbosah,East - مملكتي
3,برررررد,Basbosah,East - مملكتي
4,🤗🤗🤗🤗,Basbosah,East - مملكتي
5,في السعوديه زمان حنا نشوف بعض الدول العربيه متحررين بزود وهم ينظرون لنف...,Basbosah,East - مملكتي
7,ههههههههه الزيت 🤣🤣🤣روح تزحلق فيه,Basbosah,East - مملكتي
8,شوفوا الملعنه جايزتن له وهو مسبب لها الحزن وقاعد يتغزل بدموعها اللي نزل...,Basbosah,East - مملكتي
9,ياريتك ملبي النداء يايوسف وفاكنا,Basbosah,East - مملكتي
10,ممكن ابوها عنده زهايمر ونسى انه زوجها طيب هي عادي عندها 🤔,Basbosah,East - مملكتي


In [719]:
query_cleaned.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2705 entries, 0 to 3182
Data columns (total 3 columns):
cleaned_text    2705 non-null object
cleaned_name    2705 non-null object
cleaned_geo     2705 non-null object
dtypes: object(3)
memory usage: 84.5+ KB


### Consolidate different geo tags

In [720]:
locations = {'Saudi Arabia': ['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia', 'الخبر, المملكة العربية السعودي', 'السعودية', 'القصيم', 'تبوك, المملكة العربية السعودية', 'جدة', 'الخفجي, المملكة العربية السعودية'],
             'Kuwait': ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait', 'دولة الكويت-الفيحاء', 'Al Adailiya, Kuwait'],
             'Egypt': ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مصر', 'El Behera, Egypt', 'الجيزة, مصر', 'Elmansoura', 'El Menia, Egypt', 'El Qaliobia, Egypt', 'البحيرة, مصر', 'Domyat elgadeda', 'El Sharkia, Egypt', 'sharkia_fakous', 'Cairo, Egypt', 'Alexandria,Egypt', 'ELsharkia , Egypt', 'شمال سيناء, مصر', 'كفر الشيخ, مصر', 'Port Said, Egypt', 'Beni Suef, Egypt', 'القليوبية, مصر', 'بنها', 'Aswan', 'Beni Suef, Egypt', 'Zagazig,Egypt', 'Ismalia, Egypt', 'Kafr El Shikh, Egypt', 'القاهرة, مصر', 'البحيرة, مصر', 'live in q8 .. bas masrya', 'القاهرة, مصر', 'Domiat, Egypt', 'Alexandria, Egypt', 'Cairo, Egypt', 'El Menia', 'أسكندرية', 'Alexandria', 'tanta', 'giza,egypt', 'Giza Governorate, Egypt', 'Mansoura', 'Mansura,Egypt','المنوفية, مصر', 'مصر الاسكندرية','El Sharkia, Egypt', 'El Daqahlia, Egypt', 'Cairo -Egypt', 'Cairo , Egypt', 'Aswan, Egypt', 'Mansurah, Egypt', 'الأسكندرية, مصر', 'alexandria', 'الأقصر, مصر', 'Cairo', 'El Gharbia, Egypt', 'egypt', 'Mansoura,Egypt', 'Beni Suef, Egypt', 'Mansoura', 'El Daqahlia, Egypt', 'El Monofia, Egypt'],
             'UAE': ['United Arab Emirates', 'Dubai, United Arab Emirates', 'Dubai'],
             'Oman': ['سلطنة عمان', 'oman']
            }

In [693]:
'MAKKAH' in list(locations.values())[0]

True

In [694]:
'MAKKAH' in list(locations.values())[1]

False

In [695]:
locations.values()

dict_values([['Kingdom of Saudi Arabia', 'MAKKAH', 'saudi arabia', 'jeddah', 'مكة المكرمة', 'المملكة العربية', 'Jeddah', 'Riyadh', 'المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'الرياض, المملكة العربية السعودية', 'jeddah , saudi arabia', 'الرياض', 'آلقصيم ، بريده', 'جدة, المملكة العربية السعودية', 'المدينة المنورة, المملكة العربية السعودية', 'Riyadh, Kingdom of Saudi Arabia', 'makkah - saudi arabia', 'جده', 'Dammam', 'Dammam, Eastern', 'Al Khobar, Kingdom of Saudi Arabia', 'الخبر, المملكة العربية السعودي', 'السعودية', 'القصيم', 'تبوك, المملكة العربية السعودية', 'جدة', 'الخفجي, المملكة العربية السعودية'], ['الكويت', 'Alkuwait', 'kuwait', 'KUWAIT', 'Kuwait , Salwa', 'Qortuba, Kuwait', 'دولة الكويت', 'العاصمه, دولة الكويت', 'The Capital, Kuwait', 'Al Salam, Kuwait', 'Salwa, Kuwait', 'كوكب الشرق - Kuwait', 'دولة الكويت-الفيحاء', 'Al Adailiya, Kuwait'], ['Domiat, Egypt', 'Damitta', 'Alexandria, Egypt', 'Damitta-Egypt', 'Giza, Egypt', 'New damieta city', 'مصر', 'القاهرة, مص

In [721]:
for word in query_cleaned['cleaned_geo'].str.split('_'):
    if word[0] in list(locations.values())[0]:
        query_cleaned['cleaned_geo'].replace(word, 'Saudi Arabia', inplace=True)
    elif word[0] in list(locations.values())[1]:
            query_cleaned['cleaned_geo'].replace(word, 'Kuwait', inplace=True)
    elif word[0] in list(locations.values())[2]:
            query_cleaned['cleaned_geo'].replace(word, 'Egypt', inplace=True)
    elif word[0] in list(locations.values())[3]:
            query_cleaned['cleaned_geo'].replace(word, 'UAE', inplace=True)
    else:
        if word[0] in list(locations.values())[4]:
            query_cleaned['cleaned_geo'].replace(word, 'Oman', inplace=True)

In [722]:
query_cleaned.sample(5)

,cleaned_text,cleaned_name,cleaned_geo
2695,الوساطات الحقيقية لا تعلن والتصريحات المراد منها بطولة وانتشار لا قيمة لها,Basbosah,East - مملكتي
886,استقالة الحريري أقوى تحذير يوجه لحزب الله… آن الأوان للمخدوعين أن يقفزو...,Basbosah,East - مملكتي
882,لماذا الحب يستمر في الأفلام لأن الفيلم ساعتين بس,Basbosah,East - مملكتي
973,وفي مقابل هذا يدفع لصاحب السجل مبلغ مالي مقطوع شهري عن كل صيدلية يتم اف...,Basbosah,East - مملكتي
24,مو هاذي بالبحرين,Basbosah,East - مملكتي


### Pickle to conduct EDA in seperate notebook

In [723]:
pwd

'/home/jovyan/capstone-35/gulf_twitter_raw/gulf_timelines'

In [724]:
cd ../../gulf_twitter_pickled/

/home/jovyan/capstone-35/gulf_twitter_pickled


In [725]:
query_cleaned.to_pickle('../gulf_twitter_pickled/user_timeline_Basbosah.p')